In [11]:
# -*- encoding: utf8-*-
import requests
import time
import random
import json
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#print(json.dumps(return_json, indent=2))

In [12]:
def get_top_songs():
    top_songs = []

    headers = {'Referer': 'https://kma.kkbox.com/charts/yearly/newrelease?lang=tc&terr=tw',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
                'Content-Type': 'application/json'}
    url = 'https://kma.kkbox.com/charts/api/v1/yearly?category=297&lang=tc&limit=100&terr=tw&type=newrelease&year=2021'
    
    return_data = requests.get(url, headers=headers)
    return_json = json.loads(return_data.text)

    for song in return_json['data']['charts']['newrelease']:
        top_songs.append(re.split(' -| \(', song['song_name'])[0])
    
    return top_songs

In [19]:
# 獲取樂譜頁面
def get_pu(song_name):

    url = 'https://www.91pu.com.tw/plus/search.php?keyword=' + song_name + '&type='
    print(url)
    return_html = requests.get(url)
    soup = BeautifulSoup(return_html.text, 'html.parser')
    pu_url = 'https://www.91pu.com.tw' + soup.select('tr > td > a')[0].get('href')

    return pu_url

In [14]:
def get_chords_and_lyrics(pu_url):
    chord, lyric = [], []

    options = Options()
    options.add_argument("--disable-notifications")
    options.headless = True

    driver = webdriver.Edge(options=options)
    driver.implicitly_wait(3)
    driver.get(pu_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    tone = soup.find(id='tone_z').find_all('p')
    # print(tone)
    clean = re.compile('<.*?>')
    nbsp = re.compile(u'\xa0+')
    em = re.compile('\u2003')
    for tags in tone:
        output = tags.text
        output = re.sub(clean, '', output)
        output = re.sub(nbsp, ' ', output)
        output = re.sub(em, ' ', output)
        print(output)
        
        classes = tags.get('class')
        if 'music' in classes:
            chord.append(output)
        elif 'lyric' in classes:
            lyric.append(output)
    driver.quit()
    return chord, lyric

In [15]:
top_songs = get_top_songs()

chords = []
lyrics = []
for i in range(100):
    try:
        pu_url = get_pu(top_songs[i])
        chord, lyric = get_chords_and_lyrics(pu_url)
        chords.append(chord)
        lyrics.append(lyric)
    except:
        chords.append(['no chords'])
        lyrics.append(['no lyrics'])



[前奏] |Am |Am |G |G |F |F |C |C |
 |Dm |Dm |Am |Am |G |G |E |E |
 
|Am |Am |G |G |
 倒佇故鄉街市的中央 笑看世界咧轉踅
 |F |F |C |C |
 你是我 烏暗的心唯一的光 親像是眼前的燈火
 |Dm |Dm |Am |Am |
 原諒我這款放蕩的人 原諒我做這款夭壽的夢
 |G |G |E |E |
 每一擺你的眼神咧振動 攏予我無法分西東 哦
|Am |Am |G |G | 
 我用我的人格來保證 我是永遠袂後悔 
 |F |F |C |C |
 你是我 目前為止的愛人內底 無疑是上水的彼個
 |Dm |Dm |Am |Am |
 我其實毋是毋願離開 攏怪你行入了我的人生
 |G |G |E |E |
 無心的愛情傷害已造成 你我著愛來負責任 哦
 
 [間奏1] |Am |Am |G |G |F |F |C |C |
 |Dm |Dm |Am |Am |G |G |E |E |
 
 |Am |F |G |C |
★愛情你比我想的閣較偉大 你予我這世人最後的期待
 |Bm7-5 |E |Am |Am |
 你是我的未來 希望你會了解
 |F |G |Em |Am |F |G |
 雖然江山易改 但本性難移 愛你愛甲白目眉
 
(1)
[間奏2] |Am |Am |G |G |F |F |C |C |
 |Dm |Dm |Am |Am |G |G |E |E |
 
|Am |Am |G |G |F |F |C |C |
 倒佇故鄉街市的中央 笑看世界咧轉踅
 |Dm |Dm |Am |Am |G |G |E |E |
 我其實毋是毋願離開 攏怪你行入了我的人生 (回★)
 
(2)
 |F |G |Em |Am |F |G |
 毋管你知毋知 予你全部的愛 愛你愛甲白目眉
 
[尾奏] |Am |Am |G |G |F |F |C |C |
 |Dm |Dm |Am |Am |G |G |E |E |Am |Am |
 
 
[前奏] ||G |G |G |G || x 2
 
 |G |G |Em |Em |
 三番故里 窗外雨滴打破細碎的玻璃
 |G |G |Em |Em |
 與你相遇 是在一個單純美好的世界
 |Bm |C |G |G |
 而在變質那天 你淚流誇張情

In [23]:
f = open('lyrics.txt', "w", encoding='UTF-8')
num = 0
for i in lyrics:
    f.write(top_songs[num]+"\n")
    for j in i:
        f.write(j + ' ')
    f.write("\n")
    num += 1
f.close()

In [21]:
song_name='失重前幸福'

pu_url = get_pu(song_name)
chord, lyric = get_chords_and_lyrics(pu_url)
print(chord)
print(lyric)

https://www.91pu.com.tw/plus/search.php?keyword=失重前幸福&type=
[前奏] |Dm7 Em7 |Fm6 |Dm7 Em7 |Fm6 |
 
|Cmaj7 |Dm7 G |Cmaj7 Em7 |Dm7 G |
 記得被你拋得有多高 信任你的環抱 在我華麗翻轉後炫耀 yeah
|Cmaj7 Am7 |Dm7 G |Cmaj7 Am7 |Dm7 Fm6 |
 1.不料我們偏差了分毫 你沒把我接牢 跌落在計畫外了糟糕 ooh
▲2.自由落體撞擊的力道 我們沒計算好 眼看著目標上下顛倒 ooh
 
 |Fmaj7 E7 |Am7 － Gm7 Gm7/C |
 誰的幻想 完美的飛翔 就像練習相愛的時光
 |Fmaj7 E7 |Dm7 Fm6 |
 我重重墜下 才知道多傻 我們都會故障
 
 |C E7 |Am Am7/G |
★時間 差一點 剛好 慢一點 不妙 美麗的失誤
 |Fmaj7 － Em7 Am7 |Dm7 Fm6 |
 傷了你 傷了我 才學會彼此呵護 ooh
 |C E7 |Am Am7/G F#m7-5 － |
 愛情 缺一步 恰巧 多一步 到老 好比 這 領悟 
 |Dm7 G |Csus4 C |
 讓我們把握失重前幸福
 
(1)
[間奏] |Dm7 Em7 |Fm C |Dm7 Em7 |Fm －－ G | (回▲)
 
(2)
 |E7/B E7 |Am F#m7-5 |
 你宣告 不管疤痕 我永遠是最美的 
 |Dm7 Em7 |Fm G |
 我們找回了 重心與平衡 (回★)
 
(3)
[尾奏] |Dm7 Em7 |Cmaj7 |
 
 
 
 
['[前奏] |Dm7 Em7 |Fm6 |Dm7 Em7 |Fm6 |', '|Cmaj7 |Dm7 G |Cmaj7 Em7 |Dm7 G |', '|Cmaj7 Am7 |Dm7 G |Cmaj7 Am7 |Dm7 Fm6 |', ' |Fmaj7 E7 |Am7 － Gm7 Gm7/C |', ' |Fmaj7 E7 |Dm7 Fm6 |', ' |C E7 |Am Am7/G |', ' |Fmaj7 － Em7 Am7 |Dm7 Fm6 |', ' |C E7 |Am Am7/G F#m7-5 － |', ' |Dm7 G |Csus4 C

In [22]:

f = open('out.txt', "a", encoding='UTF-8')
f.write(song_name+"\n")
for j in chord:
    f.write(j + ' ')
f.write("\n")
f.close()